In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__


'1.5.0'

In [2]:
BATCH_SIZE=512 #大概需要2G的显存
EPOCHS=20 # 总共训练批次
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [3]:
#使用torchvision下载MNIST数据集
#download=True指定如果数据不存在先下载数据
#官方提供的数据集是用numpy array存储的
#transform会把numpy array变成torch tensor，然后把rgb值归一到指定区间
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(), #常见操作
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)


In [4]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)


In [5]:
#1. 继承nn.module这个类
#2. 定义如下两个函数
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        # Conv2d的第一个参数是输入的channel数量，第二个是输出的channel数量，第三个是kernel size
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        #由于上一层有20个channel输出，每个feature map大小为10*10，所以全连接层的输入是20*10*10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
        #最终有10类，所以最后一个全连接层输出数量是10
    #forward这个函数定义了前向传播的运算    
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out


In [6]:
#定义Loss函数和优化方法，最简单的就是使用SGD，Adam
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) # 复制到GPU/CPU
        #首先要把梯度清零，不然PyTorch每次计算梯度会累加，不清零的话第二次算的梯度等于第一次加第二次的
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        #算完loss之后进行反向梯度传播，这个过程之后梯度会记录在变量中
        loss.backward()
        # 用计算的梯度去做优化
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [10]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.117698
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.059800
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.070951

Test set: Average loss: 0.0565, Accuracy: 9805/10000 (98%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.045604
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.035453
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.043553

Test set: Average loss: 0.0442, Accuracy: 9853/10000 (99%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.026080
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.035433
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.030066

Test set: Average loss: 0.0359, Accuracy: 9893/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.016160
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.036232
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.021145

Test set: Average loss: 0.0336, Accuracy: 9885/10000 (99%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.011761
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.030418
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.021702

Test 